In [1]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

from sklearn.metrics import classification_report
from keras.optimizers import SGD,Adam
from keras.datasets import mnist
from keras.utils import np_utils
from keras import backend as K
from imutils import build_montages
import matplotlib.pyplot as plt
import numpy as np
import cv2



Using TensorFlow backend.


ModuleNotFoundError: No module named 'imutils'

In [4]:
!pip install imutils

  Created wheel for imutils: filename=imutils-0.5.3-cp37-none-any.whl size=25850 sha256=2cbc6a5c88ab51c9cb28c18e440b96eda6ced01d60c9a764b4bbe5d3d1c5ee02
  Stored in directory: /Users/ZN5997@engie.com/Library/Caches/pip/wheels/16/84/1f/bf88641293cda2c8be81a5c4b8ca973dd9125a6dc3767417fd
Successfully built imutils


In [5]:
width=28
height=28
depth=1
classes=10
chanDim = -1
 
model = Sequential()
inputShape = (height, width, depth)
# first CONV => RELU => CONV => RELU => POOL layer set

model.add(Conv2D(32, (3, 3), padding="same",
input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))

model.add(Conv2D(32, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# second CONV => RELU => CONV => RELU => POOL layer set
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))

model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# first (and only) set of FC => RELU layers
model.add(Flatten())
model.add(Dense(512))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
 
# softmax classifier
model.add(Dense(classes))
model.add(Activation("softmax"))














Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 28, 28, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
__________

In [7]:
NUM_EPOCHS = 25
INIT_LR = 1e-2
BS = 32

In [8]:
# grab the Fashion MNIST dataset (if this is your first time running
# this the dataset will be automatically downloaded)
print("[INFO] loading Fashion MNIST...")
((trainX, trainY), (testX, testY)) = mnist.load_data()
 

# if we are using "channels first" ordering, then reshape the design
# matrix such that the matrix is:
# 	num_samples x depth x rows x columns
if K.image_data_format() == "channels_first":
	trainX = trainX.reshape((trainX.shape[0], 1, 28, 28))
	testX = testX.reshape((testX.shape[0], 1, 28, 28))
 
# otherwise, we are using "channels last" ordering, so the design
# matrix shape should be: num_samples x rows x columns x depth
else:
	trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
	testX = testX.reshape((testX.shape[0], 28, 28, 1))
 
trainX =  trainX[:100]
trainY = trainY[:100]


[INFO] loading Fashion MNIST...
11493376/11490434 [==============================] - 85s 7us/step


In [9]:
# scale data to the range of [0, 1]
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0
 
# one-hot encode the training and testing labels
trainY = np_utils.to_categorical(trainY, 10)
testY = np_utils.to_categorical(testY, 10)
 
# initialize the label names
labelNames = ["0","1","2","3","4","5","6","7","8","9"]


In [10]:
# initialize the optimizer and model
print("[INFO] compiling model...")
opt = SGD(lr=INIT_LR, momentum=0.9, decay=INIT_LR / NUM_EPOCHS)

model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
 
# train the network
print("[INFO] training model...")
H = model.fit(trainX, trainY,
	validation_data=(testX, testY),
	batch_size=BS, epochs=NUM_EPOCHS)

[INFO] compiling model...


[INFO] training model...
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 100 samples, validate on 10000 samples
Epoch 1/25
100/100 [==============================] - 20s 204ms/step - loss: 3.2847 - acc: 0.1000 - val_loss: 2.8347 - val_acc: 0.2828
Epoch 2/25
100/100 [==============================] - 13s 127ms/step - loss: 1.7997 - acc: 0.4500 - val_loss: 4.8858 - val_acc: 0.4495
Epoch 3/25
100/100 [==============================] - 13s 127ms/step - loss: 0.8198 - acc: 0.7400 - val_loss: 6.8624 - val_acc: 0.4104
Epoch 4/25
100/100 [==============================] - 13s 129ms/step - loss: 0.3871 - acc: 0.8700 - val_loss: 7.6210 - val_acc: 0.4112
Epoch 5/25
100/100 [==============================] - 13s 128ms/step - loss: 0.5274 - acc: 0.8600 - val_loss: 5.7406 - val_acc: 0.5129
Epoch 6/25
100/100 [==============================] - 13s 127ms/step - loss: 0.2303 - acc: 0.9200 - val_loss: 3.3485 - val_acc: 

In [11]:
# make predictions on the test set
preds = model.predict(testX)
 
# show a nicely formatted classification report
print("[INFO] evaluating network...")
print(classification_report(testY.argmax(axis=1), preds.argmax(axis=1),
	target_names=labelNames))
 
# plot the training loss and accuracy
N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")

plt.title("Training Loss on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="lower left")
plt.savefig("plot.png")

[INFO] evaluating network...
              precision    recall  f1-score   support

           0       0.73      0.92      0.81       980
           1       0.96      0.87      0.92      1135
           2       0.64      0.77      0.70      1032
           3       0.70      0.63      0.66      1010
           4       0.46      0.80      0.58       982
           5       0.93      0.07      0.14       892
           6       0.98      0.20      0.33       958
           7       0.81      0.61      0.70      1028
           8       0.58      0.38      0.46       974
           9       0.35      0.78      0.49      1009

    accuracy                           0.61     10000
   macro avg       0.72      0.60      0.58     10000
weighted avg       0.72      0.61      0.59     10000



NameError: name 'plt' is not defined

In [12]:
import statistics
statistics.mean(H.history["val_accuracy"])

KeyError: 'val_accuracy'

In [ ]:
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")

plt.title("Training Accuracy on Mnist 100 sample")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")